In [ ]:
# song_popularity_predictor.py

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold, cross_val_score

# -----------------------------
# 1. Load Data
# -----------------------------
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Drop id column but keep for submission later
train_data = train.drop(columns=["id"])
test_data = test.drop(columns=["id"])

# -----------------------------
# 2. Feature Engineering
# -----------------------------
def add_features(df):
    df = df.copy()
    
    # duration in minutes
    df["duration_min"] = df["song_duration_ms"] / 60000
    
    # log transforms for skewed features
    for col in ["loudness", "song_duration_ms", "tempo"]:
        df[f"log_{col}"] = np.log1p(df[col].abs() + 1e-6)
    
    # ratios / interactions
    df["energy_per_dance"] = df["energy"] / (df["danceability"] + 1e-6)
    df["speech_per_liveness"] = df["speechiness"] / (df["liveness"] + 1e-6)
    df["acoustic_x_instrumental"] = df["acousticness"] * df["instrumentalness"]
    df["dance_energy"] = df["danceability"] * df["energy"]
    
    return df

train_data = add_features(train_data)
test_data = add_features(test_data)

# -----------------------------
# 3. Split Data
# -----------------------------
X = train_data.drop(columns=["song_popularity"])
y = train_data["song_popularity"]

imputer = SimpleImputer(strategy='median')  # or 'mean'
X_imputed = imputer.fit_transform(X)
X = pd.DataFrame(X_imputed, columns=X.columns)

test_data_imputed = imputer.transform(test_data)
test_data = pd.DataFrame(test_data_imputed, columns=test_data.columns)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale numerical features
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_val_scaled = scaler.transform(X_val)
# X_test_scaled = scaler.transform(test_data)

# -----------------------------
# 4. Train Multiple Models
# -----------------------------
models = {
    # "LogisticRegression": LogisticRegression(max_iter=2000),
    "RandomForest": RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=300, learning_rate=0.05),
    "XGBoost": XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, use_label_encoder=False, eval_metric="logloss"),
    "CatBoost": CatBoostClassifier(iterations=300, learning_rate=0.05, depth=6, verbose=0)
}

auc_scores = {}

print("Training models and calculating AUC scores...\n")

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba)
    auc_scores[name] = auc
    print(f"{name} AUC: {auc:.4f}")

# -----------------------------
# 5. Choose Best Model
# -----------------------------
best_model_name = max(auc_scores, key=auc_scores.get)
best_model = models[best_model_name]
print(f"\n✅ Best model: {best_model_name} (AUC = {auc_scores[best_model_name]:.4f})")

# -----------------------------
# 6. Predict on Test Data
# -----------------------------
test_predictions_proba = best_model.predict_proba(X_test_scaled)[:, 1]

# -----------------------------
# 7. Save Submission
# -----------------------------
submission = pd.DataFrame({
    "id": test["id"],
    "song_popularity": test_predictions_proba
})
submission.to_csv("submission_manual.csv", index=False)

print("\nSubmission file 'submission_manual.csv' created successfully!")
print(submission.head())


Training models and calculating AUC scores...

RandomForest AUC: 0.5717
GradientBoosting AUC: 0.5685


/home/yash/miniconda3/envs/umc/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:19:43] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost AUC: 0.5586
CatBoost AUC: 0.5670

✅ Best model: RandomForest (AUC = 0.5717)

Submission file 'submission_manual.csv' created successfully!
   id  song_popularity
0   0         0.369763
1   1         0.297793
2   2         0.318386
3   3         0.406753
4   4         0.427678


In [5]:
# song_popularity_predictor.py

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel

# -----------------------------
# 1. Load Data
# -----------------------------
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Drop id column but keep for submission later
train_data = train.drop(columns=["id"])
test_data = test.drop(columns=["id"])

# -----------------------------
# 2. Feature Engineering
# -----------------------------
def add_features(df):
    df = df.copy()
    # duration in minutes
    df["duration_min"] = df["song_duration_ms"] / 60000
    # log transforms for skewed features
    for col in ["loudness", "song_duration_ms", "tempo"]:
        df[f"log_{col}"] = np.log1p(df[col].abs() + 1e-6)
    # ratios / interactions
    df["energy_per_dance"] = df["energy"] / (df["danceability"] + 1e-6)
    df["speech_per_liveness"] = df["speechiness"] / (df["liveness"] + 1e-6)
    df["acoustic_x_instrumental"] = df["acousticness"] * df["instrumentalness"]
    df["dance_energy"] = df["danceability"] * df["energy"]
    return df

train_data = add_features(train_data)
test_data = add_features(test_data)

# -----------------------------
# 3. Prepare Data
# -----------------------------
X = train_data.drop(columns=["song_popularity"])
y = train_data["song_popularity"]

# Impute missing values
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X = pd.DataFrame(X_imputed, columns=X.columns)
test_data_imputed = imputer.transform(test_data)
test_data = pd.DataFrame(test_data_imputed, columns=test_data.columns)

# Feature selection using RandomForest
selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42), threshold="median")
selector.fit(X, y)
X_selected = selector.transform(X)
test_selected = selector.transform(test_data)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
test_scaled = scaler.transform(test_selected)

# Split for validation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# -----------------------------
# 4. Train Multiple Models
# -----------------------------
models = {
    "LogisticRegression": LogisticRegression(max_iter=2000, class_weight='balanced'),
    "RandomForest": RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=300, learning_rate=0.05),
    "XGBoost": XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, use_label_encoder=False, eval_metric="logloss"),
    "CatBoost": CatBoostClassifier(iterations=300, learning_rate=0.05, depth=6, verbose=0)
}

auc_scores = {}

print("Training models and calculating AUC scores...\n")
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba)
    auc_scores[name] = auc
    print(f"{name} AUC: {auc:.4f}")

# -----------------------------
# 5. Cross-Validation for Best Model
# -----------------------------
print("\nCross-validating best model...")
best_model_name = max(auc_scores, key=auc_scores.get)
best_model = models[best_model_name]
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_auc = cross_val_score(best_model, X_scaled, y, cv=cv, scoring='roc_auc', n_jobs=-1)
print(f"{best_model_name} mean CV AUC: {cv_auc.mean():.4f} (+/- {cv_auc.std():.4f})")

# -----------------------------
# 6. Ensemble Model
# -----------------------------
ensemble = VotingClassifier(
    estimators=[
        ('rf', models["RandomForest"]),
        ('gb', models["GradientBoosting"]),
        ('xgb', models["XGBoost"]),
        ('cat', models["CatBoost"])
    ],
    voting='soft'
)
ensemble.fit(X_scaled, y)
ensemble_pred = ensemble.predict_proba(test_scaled)[:, 1]

# -----------------------------
# 7. Predict on Test Data with Best Model
# -----------------------------
best_model.fit(X_scaled, y)
test_predictions_proba = best_model.predict_proba(test_scaled)[:, 1]

# -----------------------------
# 8. Save Submissions
# -----------------------------
submission_best = pd.DataFrame({
    "id": test["id"],
    "song_popularity": test_predictions_proba
})
submission_ensemble = pd.DataFrame({
    "id": test["id"],
    "song_popularity": ensemble_pred
})

submission_best.to_csv("submission_best.csv", index=False)
submission_ensemble.to_csv("submission_ensemble.csv", index=False)

print("\nSubmission files 'submission_best.csv' and 'submission_ensemble.csv' created successfully!")
print(submission_best.head())

Training models and calculating AUC scores...

LogisticRegression AUC: 0.5511
RandomForest AUC: 0.5669
GradientBoosting AUC: 0.5654


/home/yash/miniconda3/envs/umc/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:24:36] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost AUC: 0.5519
CatBoost AUC: 0.5681

Cross-validating best model...
CatBoost mean CV AUC: 0.5681 (+/- 0.0039)


/home/yash/miniconda3/envs/umc/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:25:35] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Submission files 'submission_best.csv' and 'submission_ensemble.csv' created successfully!
   id  song_popularity
0   0         0.361401
1   1         0.295036
2   2         0.305778
3   3         0.429407
4   4         0.498078


In [7]:
# advanced_pipeline.py
import pandas as pd
import numpy as np
from typing import Dict, Tuple, List
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.base import BaseEstimator, clone
from sklearn.utils import check_random_state

# External libs (make sure installed)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import joblib
import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 42
NFOLDS = 5

# -----------------------------
# 0. Config
# -----------------------------
TRAIN_PATH = "train.csv"
TEST_PATH = "test.csv"
LABEL = "song_popularity"
ID_COL = "id"
OUTPUT_SUBMISSION = "submission_advanced.csv"

# -----------------------------
# 1. Utility: feature engineering
# -----------------------------
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # safe guards for missing source columns
    def safe(col): return col in df.columns
    if safe("song_duration_ms"):
        df["duration_min"] = df["song_duration_ms"] / 60000.0
        df["log_song_duration_ms"] = np.log1p(df["song_duration_ms"].abs() + 1e-6)
    if safe("loudness"):
        df["log_loudness"] = np.log1p(df["loudness"].abs() + 1e-6)
    if safe("tempo"):
        df["log_tempo"] = np.log1p(df["tempo"].abs() + 1e-6)
    # ratios & interactions (guard missing columns)
    for a,b,name in [
        ("energy","danceability","energy_per_dance"),
        ("speechiness","liveness","speech_per_liveness"),
        ("acousticness","instrumentalness","acoustic_x_instrumental"),
        ("danceability","energy","dance_energy"),
    ]:
        if safe(a) and safe(b):
            # small epsilon + fillna to avoid infs
            df[name] = df[a].fillna(0.0) / (df[b].fillna(0.0) + 1e-6)
    return df

# -----------------------------
# 2. Load data and basic cleaning
# -----------------------------
def load_and_prepare(train_path: str, test_path: str, label: str, id_col: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)
    # keep ID for submission
    test_ids = test[id_col].copy()
    # drop id from feature frames
    train = train.drop(columns=[id_col])
    test = test.drop(columns=[id_col])
    # feature engineering
    train = add_features(train)
    test = add_features(test)
    return train, test, test_ids

# -----------------------------
# 3. Preprocessor: detect numeric/categorical and build ColumnTransformer
# -----------------------------
def build_preprocessor(train_df: pd.DataFrame, categorical_threshold: int = 20):
    # auto detect categorical columns (object or low unique counts)
    categorical = []
    numeric = []
    for col in train_df.columns:
        if col == LABEL: 
            continue
        if train_df[col].dtype == "object":
            categorical.append(col)
        else:
            # treat low-cardinality numeric fields as categorical (if any)
            nunique = train_df[col].nunique(dropna=True)
            if nunique <= categorical_threshold:
                categorical.append(col)
            else:
                numeric.append(col)
    # numeric pipeline: impute median -> scale
    numeric_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ])
    # categorical pipeline: impute most_freq -> one-hot (sparse)
    try:
    # Newer sklearn (>=1.4)
        categorical_pipeline = Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
        ]) if len(categorical) > 0 else None
    except TypeError:
    # Older sklearn (<1.4)
        categorical_pipeline = Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False))
        ]) if len(categorical) > 0 else None

    transformers = []
    if len(numeric) > 0:
        transformers.append(("num", numeric_pipeline, numeric))
    if categorical_pipeline is not None:
        transformers.append(("cat", categorical_pipeline, categorical))
    preprocessor = ColumnTransformer(transformers=transformers, remainder="drop", verbose_feature_names_out=False)
    return preprocessor, numeric, categorical

# -----------------------------
# 4. Define base models
# -----------------------------
def get_base_models(random_state: int = RANDOM_STATE) -> Dict[str, BaseEstimator]:
    models = {
        "logreg": LogisticRegression(max_iter=2000, random_state=random_state),
        "rf": RandomForestClassifier(n_estimators=500, max_depth=12, n_jobs=-1, random_state=random_state),
        "gb": GradientBoostingClassifier(n_estimators=400, learning_rate=0.05, max_depth=4, random_state=random_state),
        "xgb": XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=6, use_label_encoder=False, eval_metric="logloss", n_jobs=-1, random_state=random_state),
        "lgb": LGBMClassifier(n_estimators=800, learning_rate=0.05, max_depth=-1, n_jobs=-1, random_state=random_state),
        "cat": CatBoostClassifier(iterations=800, learning_rate=0.05, depth=6, verbose=0, random_state=random_state),
    }
    return models

# -----------------------------
# 5. OOF stacking helper
# -----------------------------
def get_oof_predictions(models: Dict[str, BaseEstimator], X: pd.DataFrame, y: pd.Series, preprocessor, n_splits: int = NFOLDS, random_state: int = RANDOM_STATE):
    """
    For each base model, produce out-of-fold (OOF) probability predictions for train
    and averaged predictions for validation folds and test.
    Returns:
      oof_train: DataFrame (n_samples, n_models) with OOF probs for train
      oof_val_scores: dict of AUC per model (from OOF predictions)
      fitted_models: list of the models trained on full training (optional)
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    n = X.shape[0]
    oof_train = pd.DataFrame(index=X.index)
    cv_scores = {}
    fitted_fold_models = {name: [] for name in models.keys()}  # store models for later test preds via averaging

    # Transform features once per fold with fit on fold's train
    for name, base_model in models.items():
        oof_preds = np.zeros(n, dtype=float)
        fold_aucs = []
        print(f"\n>> Generating OOF for {name}")
        for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
            X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

            # Create a pipeline (preprocessor + model)
            pipe = Pipeline([
                ("preprocessor", preprocessor),
                ("model", clone(base_model))
            ])
            pipe.fit(X_tr, y_tr)
            proba = pipe.predict_proba(X_val)[:, 1]
            oof_preds[val_idx] = proba
            auc = roc_auc_score(y_val, proba)
            fold_aucs.append(auc)
            fitted_fold_models[name].append(pipe)  # keep fold model for later
            print(f"  fold {fold}/{n_splits} AUC: {auc:.4f}")
        mean_auc = float(np.mean(fold_aucs))
        cv_scores[name] = mean_auc
        print(f"{name} OOF mean AUC: {mean_auc:.4f}")
        oof_train[name] = oof_preds
    return oof_train, cv_scores, fitted_fold_models

# -----------------------------
# 6. Train meta-model on OOF preds & evaluate
# -----------------------------
def train_meta_model(oof_train: pd.DataFrame, y: pd.Series, meta_model=None, random_state: int = RANDOM_STATE):
    if meta_model is None:
        meta_model = LogisticRegression(max_iter=2000, random_state=random_state)
    meta_model.fit(oof_train, y)
    oof_meta_proba = meta_model.predict_proba(oof_train)[:, 1]
    meta_auc = roc_auc_score(y, oof_meta_proba)
    return meta_model, meta_auc

# -----------------------------
# 7. Predict test set by averaging fold models and meta-model
# -----------------------------
def predict_test(models_fitted: Dict[str, List[Pipeline]], test_df: pd.DataFrame):
    """
    models_fitted: dict where for each model name we have a list of pipeline models (one per fold)
    Predict by averaging proba across folds for each base model.
    Returns DataFrame of shape (n_test, n_models)
    """
    test_preds = {}
    for name, pipes in models_fitted.items():
        # average predictions across fold pipelines
        p_fold = []
        for pipe in pipes:
            p = pipe.predict_proba(test_df)[:, 1]
            p_fold.append(p)
        p_arr = np.vstack(p_fold)  # (n_folds, n_test)
        avg = p_arr.mean(axis=0)
        test_preds[name] = avg
    return pd.DataFrame(test_preds, index=test_df.index)

# -----------------------------
# 8. Final pipeline putting it all together
# -----------------------------
def run_pipeline(train_path=TRAIN_PATH, test_path=TEST_PATH, do_hyperparam_search=False):
    # 1. Load
    train_df, test_df, test_ids = load_and_prepare(train_path, test_path, LABEL, ID_COL)
    X = train_df.drop(columns=[LABEL])
    y = train_df[LABEL]

    # 2. Build preprocessor
    preprocessor, numeric_cols, cat_cols = build_preprocessor(X)
    print(f"Numeric cols: {len(numeric_cols)}, Categorical cols: {len(cat_cols)}")

    # 3. Base models
    base_models = get_base_models()
    # optional quick hyperparam search for XGBoost to boost performance (keeps it short)
    if do_hyperparam_search:
        print("Running randomized search for XGBoost (quick)...")
        param_dist = {
            "n_estimators": [200, 400, 600],
            "max_depth": [4,6,8],
            "learning_rate": [0.01, 0.03, 0.05],
            "subsample": [0.6, 0.8, 1.0],
            "colsample_bytree": [0.6, 0.8, 1.0]
        }
        xgb = XGBClassifier(use_label_encoder=False, eval_metric="logloss", n_jobs=-1, random_state=RANDOM_STATE)
        pipe = Pipeline([("preprocessor", preprocessor), ("model", xgb)])
        rsearch = RandomizedSearchCV(pipe, {"model__" + k: v for k,v in param_dist.items()}, n_iter=10, cv=3, scoring="roc_auc", n_jobs=-1, random_state=RANDOM_STATE, verbose=1)
        rsearch.fit(X, y)
        best_est = rsearch.best_estimator_.named_steps["model"]
        print("XGBoost best params (subset):", rsearch.best_params_)
        base_models["xgb"] = best_est

    # 4. OOF predictions for stacking
    oof_train, cv_scores, fitted_fold_models = get_oof_predictions(base_models, X, y, preprocessor, n_splits=NFOLDS, random_state=RANDOM_STATE)

    # 5. Train meta-model on OOF preds
    meta_model = LogisticRegression(max_iter=2000, random_state=RANDOM_STATE)
    meta_model, meta_auc = train_meta_model(oof_train, y, meta_model)
    print("\nBase models CV AUCs:")
    for k,v in cv_scores.items():
        print(f"  {k}: {v:.4f}")
    print(f"\nMeta-model (stacker) OOF AUC: {meta_auc:.4f}")

    # 6. Predict on test set for each base model by averaging predictions from fold models
    test_oof_preds = predict_test(fitted_fold_models, test_df)  # DataFrame with base model columns
    # meta features for test set
    test_meta_features = test_oof_preds.copy()
    # meta prediction
    test_meta_proba = meta_model.predict_proba(test_meta_features)[:, 1]

    # 7. Simple blend: weighted average between meta and average of base preds
    base_mean = test_oof_preds.mean(axis=1).values
    final_blend = 0.6 * test_meta_proba + 0.4 * base_mean  # weights empirical; you can tune

    # 8. Print summary and save
    print("\nFinal blend created. Sample probabilities:")
    print(pd.Series(final_blend).head())

    submission = pd.DataFrame({
        ID_COL: test_ids,
        LABEL: final_blend
    })
    submission.to_csv(OUTPUT_SUBMISSION, index=False)
    print(f"\n✅ Saved submission to {OUTPUT_SUBMISSION}")

    # Save artifacts (models) - optional
    joblib.dump({"fitted_fold_models": fitted_fold_models, "meta_model": meta_model, "preprocessor": preprocessor}, "models_artifacts.joblib")
    print("Saved model artifacts to models_artifacts.joblib")

    return cv_scores, meta_auc, submission

# -----------------------------
# If run as script
# -----------------------------
if __name__ == "__main__":
    # Set do_hyperparam_search=True if you want to run a quick randomized search for XGBoost (longer).
    scores, meta_auc, submission_df = run_pipeline(do_hyperparam_search=False)
    print("\nDone.")


Numeric cols: 18, Categorical cols: 3

>> Generating OOF for logreg
  fold 1/5 AUC: 0.5434
  fold 2/5 AUC: 0.5458
  fold 3/5 AUC: 0.5657
  fold 4/5 AUC: 0.5559
  fold 5/5 AUC: 0.5387
logreg OOF mean AUC: 0.5499

>> Generating OOF for rf
  fold 1/5 AUC: 0.5545
  fold 2/5 AUC: 0.5711
  fold 3/5 AUC: 0.5800
  fold 4/5 AUC: 0.5673
  fold 5/5 AUC: 0.5626
rf OOF mean AUC: 0.5671

>> Generating OOF for gb
  fold 1/5 AUC: 0.5585
  fold 2/5 AUC: 0.5692
  fold 3/5 AUC: 0.5677
  fold 4/5 AUC: 0.5602
  fold 5/5 AUC: 0.5624
gb OOF mean AUC: 0.5636

>> Generating OOF for xgb
  fold 1/5 AUC: 0.5499
  fold 2/5 AUC: 0.5585
  fold 3/5 AUC: 0.5569
  fold 4/5 AUC: 0.5543
  fold 5/5 AUC: 0.5461
xgb OOF mean AUC: 0.5531

>> Generating OOF for lgb
[LightGBM] [Info] Number of positive: 8746, number of negative: 15254
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001713 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4626